# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

import nltk
nltk.download(['punkt', 'wordnet', 'stopwords', 'averaged_perceptron_tagger'])

import re
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier

import pickle

import warnings
import sklearn.exceptions
warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kay\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kay\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kay\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\kay\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# load data from database
engine = create_engine('sqlite:///messages.db')
df = pd.read_sql_table('messages', engine)
print(df.shape)

drop_indices = np.random.choice(df.index, int(df.shape[0]*0.98), replace=False)
df = df.drop(drop_indices)
print(df.shape)

X = df['message']
Y = df.iloc[:, -36:]
categories_names = df.iloc[:, -36:].columns


(26028, 40)
(521, 40)


### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    tokens = word_tokenize(re.sub(r"[^a-zA-Z0-9]", " ", text.lower()))
        
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    stop_words = stopwords.words("english")
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        if clean_tok not in stop_words:
            clean_tokens.append(clean_tok)
    
    return clean_tokens

for message in X.values[:5]:
    tokens = tokenize(message)
    print(message)
    print(tokens, '\n')

We need help. Carrefour has been forgotten completely. The foul odor is killing us. Just letting you know. Thanks!!
['need', 'help', 'carrefour', 'ha', 'forgotten', 'completely', 'foul', 'odor', 'killing', 'u', 'letting', 'know', 'thanks'] 

i am very happy, i hear god, religious hyme
['happy', 'hear', 'god', 'religious', 'hyme'] 

A cold front is found over Cuba this morning. It could cross Haiti tomorrow. Isolated rain showers are expected over our region tonight.
['cold', 'front', 'found', 'cuba', 'morning', 'could', 'cross', 'haiti', 'tomorrow', 'isolated', 'rain', 'shower', 'expected', 'region', 'tonight'] 

We are about 500 people. We're in Martissant 23, Village Dantes. God bless you.
['500', 'people', 'martissant', '23', 'village', 'dante', 'god', 'bless'] 

We have a commitee from Santo 15 nameAjeans: Youth Association in action for a new society that was founded in September 2009, the president is..
['commitee', 'santo', '15', 'nameajeans', 'youth', 'association', 'action', '

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=123)

In [6]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000025AF2BF7C10>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
y_pred = pd.DataFrame(pipeline.predict(X_test), columns=categories_names)
print("Accuracy:")
basic_pipeline = (y_test.reset_index(drop=True) == y_pred.reset_index(drop=True)).mean()

for i, column in enumerate(y_pred.columns):
    print(column)
    print(classification_report(y_test.iloc[:,i], y_pred.iloc[:,i]))

Accuracy:
related
              precision    recall  f1-score   support

           0       0.71      0.18      0.29        28
           1       0.81      0.98      0.89       103

    accuracy                           0.81       131
   macro avg       0.76      0.58      0.59       131
weighted avg       0.79      0.81      0.76       131

request
              precision    recall  f1-score   support

           0       0.83      1.00      0.90       104
           1       1.00      0.19      0.31        27

    accuracy                           0.83       131
   macro avg       0.91      0.59      0.61       131
weighted avg       0.86      0.83      0.78       131

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       131

    accuracy                           1.00       131
   macro avg       1.00      1.00      1.00       131
weighted avg       1.00      1.00      1.00       131

aid_related
              precision    re

In [8]:
# run prediction using X_test
y_pred = pd.DataFrame(pipeline.predict(X_test), columns=categories_names)

# calculate each evaluation score (accuracy, precision, recall, f1) for each category.
evaluate = {}
for i, column in enumerate(y_pred.columns):
    evaluate[column] = []
    evaluate[column].append(accuracy_score(y_test[column], y_pred[column]))
    evaluate[column].append(precision_score(y_test[column], y_pred[column]))
    evaluate[column].append(recall_score(y_test[column], y_pred[column]))
    evaluate[column].append(f1_score(y_test[column], y_pred[column]))

evaluate_df = pd.DataFrame(evaluate).T
evaluate_df.columns = ['accuracy', 'precision', 'recall', 'f1']

# print out for visualization
print(evaluate_df)

                        accuracy  precision    recall        f1
related                 0.809160   0.814516  0.980583  0.889868
request                 0.832061   1.000000  0.185185  0.312500
offer                   1.000000   0.000000  0.000000  0.000000
aid_related             0.648855   0.720000  0.315789  0.439024
medical_help            0.908397   0.000000  0.000000  0.000000
medical_products        0.946565   0.000000  0.000000  0.000000
search_and_rescue       0.984733   0.000000  0.000000  0.000000
security                0.984733   0.000000  0.000000  0.000000
military                0.946565   0.000000  0.000000  0.000000
child_alone             1.000000   0.000000  0.000000  0.000000
water                   0.938931   0.000000  0.000000  0.000000
food                    0.908397   1.000000  0.333333  0.500000
shelter                 0.908397   0.000000  0.000000  0.000000
clothing                0.984733   0.000000  0.000000  0.000000
money                   0.984733   0.000

### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),
    'vect__max_df': (0.5, 1.0),
    'vect__max_features': (None, 5000)
}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=3, n_jobs=1)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [10]:
cv.fit(X_train, y_train)

y_pred = pd.DataFrame(cv.predict(X_test), columns=categories_names)
print("Accuracy")
gridsearch_pipeline = (y_test.reset_index(drop=True) == y_pred.reset_index(drop=True)).mean()

for i, column in enumerate(y_pred.columns):
    print(column)
    (y_test.reset_index(drop=True) == y_pred.reset_index(drop=True)).mean()
    print(classification_report(y_test.iloc[:,i], y_pred.iloc[:,i]))

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV 1/5] END vect__max_df=0.5, vect__max_features=None, vect__ngram_range=(1, 1); total time=  12.0s
[CV 2/5] END vect__max_df=0.5, vect__max_features=None, vect__ngram_range=(1, 1); total time=  12.1s
[CV 3/5] END vect__max_df=0.5, vect__max_features=None, vect__ngram_range=(1, 1); total time=  11.7s
[CV 4/5] END vect__max_df=0.5, vect__max_features=None, vect__ngram_range=(1, 1); total time=  12.1s
[CV 5/5] END vect__max_df=0.5, vect__max_features=None, vect__ngram_range=(1, 1); total time=  12.7s
[CV 1/5] END vect__max_df=0.5, vect__max_features=None, vect__ngram_range=(1, 2); total time=  12.3s
[CV 2/5] END vect__max_df=0.5, vect__max_features=None, vect__ngram_range=(1, 2); total time=  12.7s
[CV 3/5] END vect__max_df=0.5, vect__max_features=None, vect__ngram_range=(1, 2); total time=  13.1s
[CV 4/5] END vect__max_df=0.5, vect__max_features=None, vect__ngram_range=(1, 2); total time=  12.8s
[CV 5/5] END vect__max_df=0.5, 

In [11]:
# run prediction using X_test
y_pred = pd.DataFrame(cv.predict(X_test), columns=categories_names)

# calculate each evaluation score (accuracy, precision, recall, f1) for each category.
evaluate = {}
for i, column in enumerate(y_pred.columns):
    evaluate[column] = []
    evaluate[column].append(accuracy_score(y_test[column], y_pred[column]))
    evaluate[column].append(precision_score(y_test[column], y_pred[column]))
    evaluate[column].append(recall_score(y_test[column], y_pred[column]))
    evaluate[column].append(f1_score(y_test[column], y_pred[column]))

evaluate_df = pd.DataFrame(evaluate).T
evaluate_df.columns = ['accuracy', 'precision', 'recall', 'f1']

# print out for visualization
print(evaluate_df)

                        accuracy  precision    recall        f1
related                 0.801527   0.808000  0.980583  0.885965
request                 0.847328   0.888889  0.296296  0.444444
offer                   1.000000   0.000000  0.000000  0.000000
aid_related             0.687023   0.833333  0.350877  0.493827
medical_help            0.908397   0.000000  0.000000  0.000000
medical_products        0.946565   0.000000  0.000000  0.000000
search_and_rescue       0.984733   0.000000  0.000000  0.000000
security                0.984733   0.000000  0.000000  0.000000
military                0.946565   0.000000  0.000000  0.000000
child_alone             1.000000   0.000000  0.000000  0.000000
water                   0.938931   0.000000  0.000000  0.000000
food                    0.900763   1.000000  0.277778  0.434783
shelter                 0.908397   0.000000  0.000000  0.000000
clothing                0.984733   0.000000  0.000000  0.000000
money                   0.984733   0.000

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [12]:
pipeline2 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf2', MultiOutputClassifier(AdaBoostClassifier()))
])

print(pipeline2.get_params())

parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),
    'vect__max_df': (0.5, 1.0),
    'vect__max_features': (None, 5000),
    'clf2__estimator__n_estimators': [10, 50, 100],
    'clf2__estimator__learning_rate': [0.1, 1.0, 2.0]
}

cv2 = GridSearchCV(pipeline2, param_grid=parameters, verbose=3, n_jobs=1)

cv2.fit(X_train, y_train)

y_pred2 = pd.DataFrame(cv2.predict(X_test), columns=categories_names)
print("Accuracy")
gridsearch_pipeline2 = (y_test.reset_index(drop=True) == y_pred.reset_index(drop=True)).mean()

for i, column in enumerate(y_pred2.columns):
    print(column)
    print(classification_report(y_test.iloc[:,i], y_pred2.iloc[:,i]))

{'memory': None, 'steps': [('vect', CountVectorizer(tokenizer=<function tokenize at 0x0000025AF2BF7C10>)), ('tfidf', TfidfTransformer()), ('clf2', MultiOutputClassifier(estimator=AdaBoostClassifier()))], 'verbose': False, 'vect': CountVectorizer(tokenizer=<function tokenize at 0x0000025AF2BF7C10>), 'tfidf': TfidfTransformer(), 'clf2': MultiOutputClassifier(estimator=AdaBoostClassifier()), 'vect__analyzer': 'word', 'vect__binary': False, 'vect__decode_error': 'strict', 'vect__dtype': <class 'numpy.int64'>, 'vect__encoding': 'utf-8', 'vect__input': 'content', 'vect__lowercase': True, 'vect__max_df': 1.0, 'vect__max_features': None, 'vect__min_df': 1, 'vect__ngram_range': (1, 1), 'vect__preprocessor': None, 'vect__stop_words': None, 'vect__strip_accents': None, 'vect__token_pattern': '(?u)\\b\\w\\w+\\b', 'vect__tokenizer': <function tokenize at 0x0000025AF2BF7C10>, 'vect__vocabulary': None, 'tfidf__norm': 'l2', 'tfidf__smooth_idf': True, 'tfidf__sublinear_tf': False, 'tfidf__use_idf': Tru

In [13]:
# run prediction using X_test
y_pred = pd.DataFrame(cv2.predict(X_test), columns=categories_names)

# calculate each evaluation score (accuracy, precision, recall, f1) for each category.
evaluate = {}
for i, column in enumerate(y_pred.columns):
    evaluate[column] = []
    evaluate[column].append(accuracy_score(y_test[column], y_pred[column]))
    evaluate[column].append(precision_score(y_test[column], y_pred[column]))
    evaluate[column].append(recall_score(y_test[column], y_pred[column]))
    evaluate[column].append(f1_score(y_test[column], y_pred[column]))

evaluate_df = pd.DataFrame(evaluate).T
evaluate_df.columns = ['accuracy', 'precision', 'recall', 'f1']

# print out for visualization
print(evaluate_df)

                        accuracy  precision    recall        f1
related                 0.763359   0.795082  0.941748  0.862222
request                 0.832061   0.692308  0.333333  0.450000
offer                   1.000000   0.000000  0.000000  0.000000
aid_related             0.671756   0.818182  0.315789  0.455696
medical_help            0.908397   0.500000  0.083333  0.142857
medical_products        0.946565   0.000000  0.000000  0.000000
search_and_rescue       0.984733   0.000000  0.000000  0.000000
security                0.984733   0.000000  0.000000  0.000000
military                0.938931   0.000000  0.000000  0.000000
child_alone             1.000000   0.000000  0.000000  0.000000
water                   0.923664   0.416667  0.625000  0.500000
food                    0.900763   0.619048  0.722222  0.666667
shelter                 0.916031   0.600000  0.250000  0.352941
clothing                0.938931   0.000000  0.000000  0.000000
money                   0.946565   0.000

In [14]:
accuracy_df = pd.concat([basic_pipeline, gridsearch_pipeline, gridsearch_pipeline2], axis=1)
accuracy_df.columns = ['basic ML', 'GridSearchCV', 'GridSearchCV2']

print(accuracy_df.mean())
print(accuracy_df.median())
print(accuracy_df.min())
print(accuracy_df.max())

basic ML         0.930874
GridSearchCV     0.932782
GridSearchCV2    0.932782
dtype: float64
basic ML         0.946565
GridSearchCV     0.946565
GridSearchCV2    0.946565
dtype: float64
basic ML         0.648855
GridSearchCV     0.687023
GridSearchCV2    0.687023
dtype: float64
basic ML         1.0
GridSearchCV     1.0
GridSearchCV2    1.0
dtype: float64


### 9. Export your model as a pickle file

In [15]:
pickle.dump(pipeline2, open('model', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.